# Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

# local packages packages
from utils import Utils
from data_utils import Data

## Getting to know the data

In [ ]:
# load data:
data = Data(Utils.read_config_for_env(config_path='../config/config.yml'))

In [ ]:
data.shop_list.info()

In [ ]:
data.item_list.info()

In [ ]:
data.category_list.info()

In [ ]:
data.transactions.info(show_counts=True)

What is price? is it unit price or total price (=unit price * amount). A high correlation between amount and price would indicate that the price is total price.

In [ ]:
plt.plot(data.transactions.amount, data.transactions.price, 'o')

No correlation. From this we can conclude that price is unit price. But it might be good to double check with the customer.

In [ ]:
print("Shapes of Each Table:")
print(f"Items table: {data.item_list.shape}")
print(f"Shops table: {data.shop_list.shape}")
print(f"Categories table: {data.category_list.shape}")
print(f"Transactions Table: {data.transactions.shape}")

In [ ]:
print("Unique count of items, shops, categories:")
print(f"Unique item count: {data.item_list.item_id.unique().shape[0]}")
print(f"Unique shop count: {data.shop_list.shop_id.unique().shape[0]}")
print(f"Unique category count: {data.category_list.item_category_id.unique().shape[0]}")

In [ ]:
print("Representation of the transaction data:")
print(f'Unique items in transactions: {data.transactions.item_id.unique().shape[0]}')
print(f'Unique shops in transactions: {data.transactions.shop_id.unique().shape[0]}')

First insights:
- no missing values at the first sight, however they may be encoded as -99, etc
- some items are not represented in the transactions data, which means they were never bought. This is entirely plausible, but this means we will not be able to make predictions for these items, as long as we don't have any other data source on what items are available in which shops.
- 1 shop (out of 60) is not represented in the shops data, which means we probably don't have data for this shop. This means we don't have transaction data for this shop, we can't make predictions for this shop, since we don't know what items are available in this shop.
- Item categories (from item_list table) seems like a potential feature we can use for predictions.
- Month data, extracted from the transactions.dates should be used as a predictive feature, we have to look closer to decide whether years is a feature or not

## Merge tables to ease analysis and visualisation

In [ ]:

# create a combined table to ease data processing and visualisation
data_merged = data.merge_data()
# create date objects and add month and year
data_merged = data.handle_dates(data_merged)
data_merged.info(show_counts=True)

## Cleaning implausible values and outliers

### Date

In [ ]:
print(f"Date range: {data_merged.date.min()}, {data_merged.date.max()}")

### Price

In [ ]:
data_merged.price.describe()

In [ ]:
data_merged[data_merged.price<0]

For price, -1 is certainly an implausible value. We should remove (and impute?) such rows where price<0

In [ ]:
plt.boxplot(data_merged.price)

That point seems like an outlier, let's take a closer look at transactions where price>100K.

In [ ]:
pd.set_option('display.max_rows', None)
data_merged[data_merged.price>100000]

>100000 (Euros?) for software like photoshop, or items like xbox and PS (amount=1) look suspicious, but given that these shops seem to be from russia, it is possible that the currency is in Rubles (this is something we should clarify with the customer), and considering that 1 Ruble = 0.010 Euros, these correspond to about 1000 Euros, which starts becoming plausible. But what about that most expensive item shown in the diagram?

In [ ]:
data_merged[data_merged.price>200000]

It turns out, Radmin is again some software, for which a price of a presumable 10.000 Euros is unreasonable, but the '522 persons' in the item name make it again difficult to judge. There can be outliers in the lower range too.
Let's eliminate the rows with significanlty deviating prices (e.g., mean -/+ 3*std). At a later stage, we can try to eliminate based on statistics of the cateogories or individual items. 

### Amount

In [ ]:
data_merged.amount.describe()

In [ ]:
data_merged[data_merged.amount<-5]

In [ ]:
data_merged[data_merged.amount<0].shape

Here, negative values might be plausible, as these may represent returns. This is something to check with the customer, but for now, we will assume that we are not interested in these data, and will remove them. Predicting 'returns' might be anyway outside of the scope of the current project.

In [ ]:
plt.boxplot(data_merged.amount)

In [ ]:
data_merged[data_merged.amount>500]

Some of these may seem ok'ish (eg, Ticket), but some are looking definitely strange (e.g., Grand Theft Auto).
Similar to the price outlier removing logic, let's eliminate the rows with significanlty deviating amounts. At a later stage, we can try to eliminate based on statistics of the cateogories or individual items.

### Cleaning

In [ ]:
data_cleaned = data.clean_data(data_merged)

Here, we used the mean +/- 3*std outlier detection strategy, and it turns out, we lost a considerable amount (1.7%) of data. And given that the higher amounts of sales are valuable to predict, so that shops are ready for such singular events, this outlier removal strategy might be too aggressive. This should be definitely revisited at a later stage, probably involving the customer into the decision.

In [ ]:
data_cleaned.describe()

In [ ]:
plt.boxplot(data_cleaned.price)

In [ ]:
plt.boxplot(data_cleaned.amount)

## Looking for patterns

### Time
Let's look at the distribution of transaction counts over time, to see if there is any anomaly:

In [ ]:
daily_total_counts = data_cleaned.groupby('date').size()
daily_total_counts = daily_total_counts.reset_index(name='count')

px.line(daily_total_counts, x='date', y='count', title='Count of Transactions for Each Date')

In [ ]:
daily_total_amounts = data_cleaned[['date', 'amount']].groupby(['date'], group_keys=True).sum()
daily_total_amounts = daily_total_amounts.reset_index()

px.line(daily_total_amounts, x='date', y='amount', title='Total Amount of Sales for Each Date')

 It seems like there is a downward trend in both transaction counts and sale amounts over time. It might be good to use a model that can take this into account. We can also see the christmas/sylvester peaks. For monthly sales, these daily features are not important, but these probably reflect to the monthly sales numbers: 

In [ ]:
monthly_total_amounts = data_cleaned[['month', 'amount']].groupby(['month']).sum()
monthly_total_amounts = monthly_total_amounts.reset_index()

px.bar(monthly_total_amounts, x='month', y='amount', title='Total Amount of Sales for Each Month')

From this chart, we see that there is an obvious seasonal component, with higher sales during during winter months in general, starting with December.

In [ ]:
dayofweek_total_amounts = data_cleaned[['dayofweek', 'amount']].groupby(['dayofweek']).sum()
dayofweek_total_amounts = dayofweek_total_amounts.reset_index()
# dayofweek_total_amounts['dayofweek'] = dayofweek_total_amounts['dayofweek'].dt.strftime("%A")
fig = px.bar(dayofweek_total_amounts, x='dayofweek', y='amount', title='Total Amount of Sales for Days of the Week')
fig.update_layout(xaxis=dict(tickmode='array', tickvals=[0, 1, 2, 3, 4, 5, 6], ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']))
fig.show()

We see that the sales increase during the weekends. For making predictions on a daily basis, this should be taken into account.

### Other factors
Let's try to find relationships between sales and other potential factors, such as shops, categories, and price

In [ ]:
shop_amounts = data_cleaned[['shop_id', 'amount']].groupby(['shop_id']).sum()
shop_amounts = shop_amounts.reset_index()

px.bar(shop_amounts, x='shop_id', y='amount', title='Total Amount of Sales for Each Shop')

In [ ]:
shop_amounts_monthly = data_cleaned[['date', 'shop_id', 'amount']]
shop_amounts_monthly['Period'] = shop_amounts_monthly['date'].dt.to_period('M').astype('str')
shop_amounts_monthly = shop_amounts_monthly[['Period', 'shop_id', 'amount']].groupby(['Period', 'shop_id']).sum()
shop_amounts_monthly = shop_amounts_monthly.reset_index()
px.line(shop_amounts_monthly, x='Period', y='amount', color='shop_id', title='Total Sales Amount Across Time for Each Shop')

The total amount of sales vary strongly across shops. The trends we've seen before seem to apply for all shops.

In [ ]:
category_amounts = data_cleaned[['item_category_name', 'amount']].groupby(['item_category_name']).sum()
category_amounts = category_amounts.sort_values(by='amount', ascending=False).reset_index()
category_amounts = category_amounts.loc[category_amounts['amount']>10000,:]
fig = px.bar(category_amounts, x='amount', y='item_category_name', color='item_category_name', title='Total Amount of Sales for Each Category (with sale amount >10,000)')
fig.update_layout(showlegend=False)

We can try to use category as a feature, although it is questionable whether it would add value.

What about Price? It is well possible that the price has an effect on the sales, for each shop and item pair. In the prediciton, we don't have information on the price to be set for each shop and item pair for the month to be predicted. For now we could calculate the average price of items in each shop, but it is questionable that this would add value to our model, since the shop_id's and prices would be one to one mappable. In a later iteration, we can talk with the customer and ask if they can provide the price of each item to be set for each prediction month.